## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-10-10-06-22 +0000,nyt,Eddie Bauer Files for Bankruptcy,https://www.nytimes.com/2026/02/10/business/ed...
1,2026-02-10-10-04-30 +0000,bbc,Palestinians say new Israeli measures in West ...,https://www.bbc.com/news/articles/cvg5x2l3v0qo...
2,2026-02-10-10-04-26 +0000,bbc,'Mum is still out there' - Savannah Guthrie is...,https://www.bbc.com/news/articles/c4geqzqd8qeo...
3,2026-02-10-10-04-04 +0000,nyt,Top Immigration Officials Set to Testify Befor...,https://www.nytimes.com/2026/02/10/us/politics...
4,2026-02-10-10-03-30 +0000,nyt,"Many Residents in Minneapolis, Tempered by ICE...",https://www.nytimes.com/2026/02/10/us/twin-cit...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2511/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
86,trump,28
296,epstein,20
5,new,16
191,starmer,14
628,bowl,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
19,2026-02-10-09-57-46 +0000,nypost,Trump says he will block US-Canada Bridge unle...,https://nypost.com/2026/02/10/us-news/trump-sa...,82
102,2026-02-10-01-54-00 +0000,wsj,Trump Threatens to Block Opening of New Bridge...,https://www.wsj.com/world/americas/trump-threa...,81
276,2026-02-09-15-32-32 +0000,nypost,"Ghislaine Maxwell offers to clear Trump, Clint...",https://nypost.com/2026/02/09/us-news/ghislain...,79
112,2026-02-10-01-13-00 +0000,wsj,The White House is at loggerheads with Congres...,https://www.wsj.com/politics/policy/proposed-b...,77
297,2026-02-09-13-23-19 +0000,nyt,Trump Criticizes Bad Bunny’s Super Bowl Halfti...,https://www.nytimes.com/2026/02/09/us/trump-ba...,74


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
19,82,2026-02-10-09-57-46 +0000,nypost,Trump says he will block US-Canada Bridge unle...,https://nypost.com/2026/02/10/us-news/trump-sa...
122,67,2026-02-10-00-44-27 +0000,nypost,Epstein victims use Super Bowl commercial to p...,https://nypost.com/2026/02/09/us-news/epstein-...
41,64,2026-02-10-08-32-51 +0000,nypost,Nancy Guthrie was supposed to go to pal’s hous...,https://nypost.com/2026/02/10/us-news/nancy-gu...
35,46,2026-02-10-08-46-21 +0000,wsj,Stock Market Today: Japanese Stocks Scale New ...,https://www.wsj.com/livecoverage/stock-market-...
269,42,2026-02-09-16-00-19 +0000,cbc,Canada's major airlines wind down flights to C...,https://www.cbc.ca/news/canada/travel-to-cuba-...
125,39,2026-02-10-00-25-30 +0000,nyt,"San Francisco Teachers Begin Strike, Closing S...",https://www.nytimes.com/2026/02/09/us/san-fran...
310,37,2026-02-09-12-26-10 +0000,bbc,Chris Mason: Starmer's predicament is dire and...,https://www.bbc.com/news/articles/ce3g8e7n07jo...
112,36,2026-02-10-01-13-00 +0000,wsj,The White House is at loggerheads with Congres...,https://www.wsj.com/politics/policy/proposed-b...
170,33,2026-02-09-22-08-06 +0000,nyt,How Ilia Malinin and Team U.S.A. Held Off Japa...,https://www.nytimes.com/2026/02/08/world/how-t...
110,30,2026-02-10-01-25-52 +0000,latimes,Federal judge strikes down California mask ban...,https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
